## Initial Refinement: choose which columns to keep

In [1]:
import pandas as pd

## Overview

The collision dataset from 2023 is quite large: there are 104257 rows. 

**Null values?**

At first glance it looks like there are no null values but this is because of the way that the data has been encoded (not because all values are relevant or meaningful).

**Categorical or Numerical data?**

Although there are a lot of numerical data types, most of these encode categories.


In [2]:
df_collision = pd.read_csv("./data/dft-road-casualty-statistics-collision-2023.csv", 
                          date_format="%d/%m/%Y", parse_dates=["date"], dtype={"accident_index": str, "accident_reference":str})
df_collision.head()

,accident_index,accident_year,accident_reference,location_easting_osgr,location_northing_osgr,longitude,latitude,police_force,accident_severity,number_of_vehicles,...,light_conditions,weather_conditions,road_surface_conditions,special_conditions_at_site,carriageway_hazards,urban_or_rural_area,did_police_officer_attend_scene_of_accident,trunk_road_flag,lsoa_of_accident_location,enhanced_severity_collision
0,2023010419171,2023,010419171,525060.0,170416.0,-0.202878,51.418974,1,3,1,...,4,8,2,0,0,1,1,2,E01003383,-1
1,2023010419183,2023,010419183,535463.0,198745.0,-0.042464,51.671155,1,3,3,...,4,1,1,0,0,1,1,2,E01001547,-1
2,2023010419189,2023,010419189,508702.0,177696.0,-0.435789,51.487777,1,3,2,...,4,1,1,0,0,1,1,2,E01002448,-1
3,2023010419191,2023,010419191,520341.0,190175.0,-0.263972,51.597575,1,3,2,...,4,9,1,0,0,1,1,2,E01000129,-1
4,2023010419192,2023,010419192,527255.0,176963.0,-0.168976,51.477324,1,3,2,...,4,1,1,0,0,1,1,2,E01004583,-1


In [3]:
df_collision.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104258 entries, 0 to 104257
Data columns (total 37 columns):
 #   Column                                       Non-Null Count   Dtype         
---  ------                                       --------------   -----         
 0   accident_index                               104258 non-null  object        
 1   accident_year                                104258 non-null  int64         
 2   accident_reference                           104258 non-null  object        
 3   location_easting_osgr                        104246 non-null  float64       
 4   location_northing_osgr                       104246 non-null  float64       
 5   longitude                                    104246 non-null  float64       
 6   latitude                                     104246 non-null  float64       
 7   police_force                                 104258 non-null  int64         
 8   accident_severity                            104258 non-null  in

### Interpreting categorical variables

The original dataset encodes its categorical variables as numbers. For ease of use, they should be converted into readable strings. 

There is a [Road Safety Open Dataset Guide](https://data.dft.gov.uk/road-accidents-safety-data/dft-road-casualty-statistics-road-safety-open-dataset-data-guide-2023.xlsx) that provides the key for these values and can be used as a mapper

In [4]:
df_interp = pd.read_excel("./data/dft-road-casualty-statistics-road-safety-open-dataset-data-guide-2023.xlsx")

df_collision_key = df_interp[df_interp["table"] == "accident"]
df_collision_key.tail()

,table,field name,code/format,label,note
1381,accident,trunk_road_flag,1,Trunk (Roads managed by Highways England),NaN
1382,accident,trunk_road_flag,2,Non-trunk,NaN
1383,accident,trunk_road_flag,-1,Data missing or out of range,NaN
1384,accident,lsoa_of_collision_location,NaN,NaN,England and Wales only. See Office for Nationa...
1385,accident,lsoa_of_accident_location,NaN,NaN,England and Wales only. See Office for Nationa...


In [5]:
categorical_columns = ["police_force","accident_severity",
                       "day_of_week", "local_authority_district",
                       "first_road_class","road_type","junction_detail",
                       "junction_control", "second_road_class",
                       "pedestrian_crossing_human_control","pedestrian_crossing_physical_facilities",
                       "light_conditions", "weather_conditions", "road_surface_conditions",
                       "special_conditions_at_site", "carriageway_hazards","urban_or_rural_area",
                       "did_police_officer_attend_scene_of_accident","trunk_road_flag"]

categorical_columns_mapping = {}

for col in categorical_columns:
    filtered_key = df_collision_key[df_collision_key["field name"] == col]
    categorical_columns_mapping[col] = pd.Series(filtered_key["label"].values,index=filtered_key["code/format"]).to_dict()
    
categorical_columns_mapping

{'police_force': {1: 'Metropolitan Police',
  3: 'Cumbria',
  4: 'Lancashire',
  5: 'Merseyside',
  6: 'Greater Manchester',
  7: 'Cheshire',
  10: 'Northumbria',
  11: 'Durham',
  12: 'North Yorkshire',
  13: 'West Yorkshire',
  14: 'South Yorkshire',
  16: 'Humberside',
  17: 'Cleveland',
  20: 'West Midlands',
  21: 'Staffordshire',
  22: 'West Mercia',
  23: 'Warwickshire',
  30: 'Derbyshire',
  31: 'Nottinghamshire',
  32: 'Lincolnshire',
  33: 'Leicestershire',
  34: 'Northamptonshire',
  35: 'Cambridgeshire',
  36: 'Norfolk',
  37: 'Suffolk',
  40: 'Bedfordshire',
  41: 'Hertfordshire',
  42: 'Essex',
  43: 'Thames Valley',
  44: 'Hampshire',
  45: 'Surrey',
  46: 'Kent',
  47: 'Sussex',
  48: 'City of London',
  50: 'Devon and Cornwall',
  52: 'Avon and Somerset',
  53: 'Gloucestershire',
  54: 'Wiltshire',
  55: 'Dorset',
  60: 'North Wales',
  61: 'Gwent',
  62: 'South Wales',
  63: 'Dyfed-Powys',
  91: 'Northern',
  92: 'Grampian',
  93: 'Tayside',
  94: 'Fife',
  95: 'Lothi

## Drop columns

There are several columns in our dataset that I have decided to drop completely.

The reasoning behind this is detailed below.

After dropping these columns I will save a new csv and use this for the next stage of processing.

### 1. Drop deprecated columns with no values

`local_authority_district` refers to the local authority district but has been superseded by `local_authority_ons_district`. The local authority district is a geographical area that a local government body is responsible for.

We can see that all values are `-1` i.e.  they are all deprecated so we will drop this column

In [6]:
df_collision["local_authority_district"].value_counts()

local_authority_district
-1    104258
Name: count, dtype: int64

In [7]:
categorical_columns_mapping["local_authority_district"].get(-1)

'Code deprecated'

In [8]:
del df_collision["local_authority_district"]

### 2. Drop id columns and year

`accident_reference` is the police reference of the collision. It is not unique outside of the year. We have another identifier, `accident_index`, which is a unique reference for collisions across years. We don't need these ids so we can drop both `accident_index` and `accident_reference`.

All our data is from 2023, so we can drop the `accident_year`.

In [9]:
del df_collision["accident_year"]
del df_collision["accident_reference"]
del df_collision["accident_index"]

### 3. Drop coordinate columns

There are two coordinate systems in this dataset:

`latitude` and `longitude` provide the coordinates of the location of the collision. 

`location_easting_osgr` and `location_northing_osgr` also provide the Eastings and Northings coordinates of the location of the collision on a map. Eastings and Northings are a coordinate system used in the United Kingdom which provide the same level of pinpoint information.


#### Issues with coordinates for ML models

1. Geographic Non-Linearity

Latitude and longitude represent locations on the Earth's surface and are based on the Earth's spherical shape: this is not a plane and distances between points do not behave in a linear fashion e.g. the distance covered by 1 degree of longitude is 111km at the equator and this gradually shrinks as you near the poles to zero. This non-linearity can make latitude and longitude tricky for algorithms that assume a more regular, linear relationship between input features and output.

Eastings and Northings also suffer from this problem. They are projected coordinates that often use a cartesian grid that maps the Earth to a flat plane, making the distances and spatial relationships more linear and easier to work with than latitude and longitude.

2. Dimensionality
   
Latitude and longitude/ Eastings and northing are 2D features but spatial relationships often involve complex dependencies that these coordinates don't capture.

4. Overfitting
   
Raw coordinates are unique identifiers. They might make your model memorize specific details which could lead to overfitting, especially when your dataset is large.

**Solutions**

The advice to counter these geographic non-linearity is:
- cluster coordinates i.e. extract clusters/regions from them. 
- create new spatial features from them e.g. distance to nearest city centre

**Conclusion**

We already have administrative area categories in our dataset that provide clusters, creating an additional cluster is overkill right now.  

We have no reason in particular to think that the severity of a collision is linked to other spatial features like distance from a certain place. There are other features in our dataset that capture this idea e.g. if there is a junction nearby, if there is a pedestrian crossing nearby.

As such, we drop this data.

In [10]:
del df_collision["latitude"]
del df_collision["longitude"]
del df_collision["location_easting_osgr"]
del df_collision["location_northing_osgr"]

### 4. Drop additional Severity column

There are two columns that capture the severity of the collision: `accident_severity` and `enhanced_severity_collision`. `enhanced_severity_collision` was introduced more recently and has been included in the data when it was captured.

We do not need two competing `severity` measures. Since `accident_severity` is complete (there are no missing values) whereas 46% of the `enhanced_severity_collision` is missing, we will use `accident_severity` and delete `enhanced_severity_collision`.

In [11]:
categorical_columns_mapping["accident_severity"]

{1: 'Fatal', 2: 'Serious', 3: 'Slight'}

In [12]:
df_collision["accident_severity"].value_counts(normalize=True)

accident_severity
3    0.760594
2    0.224808
1    0.014598
Name: proportion, dtype: float64

In [13]:
enhanced_key = df_collision_key[df_collision_key["field name"] == "enhanced_collision_severity"]
categorical_columns_mapping["enhanced_severity_collision"] = pd.Series(enhanced_key["label"].values,index=enhanced_key["code/format"]).to_dict()
categorical_columns_mapping["enhanced_severity_collision"] 

{1: 'Fatal',
 5: 'Very Serious',
 6: 'Moderately Serious',
 7: 'Less Serious',
 3: 'Slight',
 -1: 'Data missing or out of range'}

In [14]:
df_collision["enhanced_severity_collision"].value_counts(normalize=True)

enhanced_severity_collision
-1    0.462708
 3    0.386656
 7    0.084454
 6    0.031432
 5    0.025619
 1    0.009131
Name: proportion, dtype: float64

In [15]:
del df_collision["enhanced_severity_collision"]

### 5. Drop junction related columns

`junction_detail`: Junction is defined as a place where two or more roads meet. If the collision happened at a junction, this field identifies the type of the junction.

Not all accidents occured at a junction e.g. in our data 40% falls into the category 'Not at junction or within 20 metres'.

`junction_control`, `second_road_class` and  `second_road_number` are only populated for "junction collisions". More than 40% of the values in the `junction_control`, `second_road_class` and `second_road_number` contain missing values. 

I will make use of `junction_detail` but drop `junction_control`, `second_road_class` and `second_road_number`: if junction collision are more likely to result in severe accidents, we can get this from `junction_detail` alone. 


In [16]:
categorical_columns_mapping["junction_detail"]

{0: 'Not at junction or within 20 metres',
 1: 'Roundabout',
 2: 'Mini-roundabout',
 3: 'T or staggered junction',
 5: 'Slip road',
 6: 'Crossroads',
 7: 'More than 4 arms (not roundabout)',
 8: 'Private drive or entrance',
 9: 'Other junction',
 99: 'unknown (self reported)',
 -1: 'Data missing or out of range'}

In [17]:
df_collision["junction_detail"].value_counts(normalize=True)

junction_detail
 0     0.412227
 3     0.278530
 6     0.091187
 1     0.072369
 9     0.059305
 99    0.022991
 8     0.021706
 7     0.015673
 2     0.015183
 5     0.010819
-1     0.000010
Name: proportion, dtype: float64

In [18]:
categorical_columns_mapping["junction_control"]

{0: 'Not at junction or within 20 metres',
 1: 'Authorised person',
 2: 'Auto traffic signal',
 3: 'Stop sign',
 4: 'Give way or uncontrolled',
 -1: 'Data missing or out of range',
 9: 'unknown (self reported)'}

In [19]:
df_collision["junction_control"].value_counts(normalize=True)

junction_control
 4    0.433617
-1    0.420064
 2    0.111732
 9    0.021332
 3    0.007318
 1    0.005937
Name: proportion, dtype: float64

In [20]:
categorical_columns_mapping["second_road_class"]

{0: 'Not at junction or within 20 metres',
 1: 'Motorway',
 2: 'A(M)',
 3: 'A',
 4: 'B',
 5: 'C',
 6: 'Unclassified',
 9: 'Unknown (self rep only)',
 -1: 'Data missing or out of range'}

In [21]:
df_collision["second_road_class"].value_counts(normalize=True)

second_road_class
 6    0.418155
 0    0.412074
 3    0.098573
 4    0.038414
 5    0.028564
 1    0.002993
-1    0.000806
 2    0.000422
Name: proportion, dtype: float64

In [22]:
second_road_number_key = df_collision_key[df_collision_key["field name"] == "second_road_number"]
categorical_columns_mapping["second_road_number"] = pd.Series(second_road_number_key["label"].values,index=second_road_number_key["code/format"]).to_dict()
categorical_columns_mapping["second_road_number"] 

{'1 to 9999': 'Number range',
 '-1': 'Unknown',
 0: 'first_road_class is C or Unclassified. These roads do not have official numbers so recorded as zero '}

In [23]:
df_collision["second_road_number"].value_counts(normalize=True).head()

second_road_number
 0     0.446719
-1     0.412880
 1     0.001583
 4     0.001448
 38    0.001324
Name: proportion, dtype: float64

In [24]:
del df_collision["junction_control"]
del df_collision["second_road_number"]
del df_collision["second_road_class"]

### 6. Drop first_road_class

There are a number of fields that provide a description of the road.

`first_road_class` describes the importance of the road (Motorways and A roads are major roads, B connect different areas, C and "undefined" are local roads.  

`first_road_number` is the number of the road the collision took place on. 

The road numbering system in the UK is based on a hub-and-spoke system. London and Edinburgh are the two major hubs. The first digit of a road's number indicates its location in a zone. The length of a number can indicate the road's importance. Lower numbers originate closer to London/Edinburgh than higher numbered ones. See [Great Britain Road Numbering Scheme].

This number could give us a sense of the importance of the road and its distance from a major hub but around 37% of the dataset categorises this field as `0`: C roads and unclassified roads don't have road numbers. 

The zero classification is a little tricky: I did consider imputing this value e.g. assigning them all to be 99999 but this implies that the number says something about the class of the road alone. It also says something about how far it is from a hub and this imputation wouldn't account for this. I also considered finding the maximum first_road_number in each police_force area: I thought this would give a sense of distance from a hub while retaining the sense that this is a minor road. However since both of these concepts are accounted for in `first_road_class` and `police_force` it didn't seem meaningful to attempt to add this. 

This data field is somewhat unclear and arguably incomplete. The information it might provide e.g. location, importance are already captured in other data fields (`police_force` and `first_road_class`).

For this reason I have decided to drop it.

In [25]:
categorical_columns_mapping["first_road_class"]

{1: 'Motorway',
 2: 'A(M)',
 3: 'A',
 4: 'B',
 5: 'C',
 6: 'Unclassified',
 -1: 'Data missing or out of range'}

In [26]:
first_road_number_key = df_collision_key[df_collision_key["field name"] == "first_road_number"]
categorical_columns_mapping["first_road_number"] = pd.Series(first_road_number_key["label"].values,index=first_road_number_key["code/format"]).to_dict()
categorical_columns_mapping["first_road_number"] 

{'1 to 9999': 'Number range',
 '-1': 'Unknown',
 0: 'first_road_class is C or Unclassified. These roads do not have official numbers so recorded as zero '}

In [27]:
# Find the road numbers for C and unclassified roads
df_collision[df_collision["first_road_class"].isin([5,6])]["first_road_number"].value_counts()

first_road_number
0    41911
Name: count, dtype: int64

In [28]:
26884/len(df_collision)

0.2578603080818738

In [29]:
del df_collision["first_road_number"]

### 7. Drop categorical columns with incomplete categories

Much of our data is categorical but some of these categories are incomplete i.e. not all the possible categories exist in our dataset.

The [Road Safety Open Dataset Guide](https://data.dft.gov.uk/road-accidents-safety-data/dft-road-casualty-statistics-road-safety-open-dataset-data-guide-2023.xlsx) provides details of all the possible category values for each field and we can use this to see if our current dataset contains them all.

*Why are incomplete categories a problem?*

If we proceed to train our model with this dataset, the model will learn to predict the categories we do have well but it won't know how to handle the 16.63% that are missing.

An obvious solution to this is to get more data. However, adding more data does not guarantee that a category will be complete. (I did play around with this possibility for other years of the dataset but was not able to resolve the problem).  

In [30]:
def get_info_on_incomplete_categories(field_name, additional_filter = None, accident_key_field ="code/format" ):
    all_possible = df_collision_key[df_collision_key["field name"] == field_name]
    
    if additional_filter is not None:
        all_possible = all_possible[additional_filter]

    all_possible = set(all_possible[accident_key_field].values)

    exist_in_df_collision = set(df_collision[field_name].values)

    no_missing = len(all_possible - exist_in_df_collision)

    percentage_missing = round((no_missing/len(all_possible)) * 100, 2)

    return len(all_possible), no_missing, percentage_missing

**Location category: Local authority ONS District**

`local_authority_ons_district` refers to the local authority district. This is a geographical area that a local government body is responsible for.

Our current dataset has missing `local_authority_ons_district` categories: 16.63% are missing. 

This is quite a significant percentage. As such we will remove this from our dataset.

In [31]:
field_name = "local_authority_ons_district"
additional_filter = df_collision_key["code/format"] != -1
get_info_on_incomplete_categories(field_name, additional_filter)

/var/folders/bz/vfyjwrvn5lzbwwdbbbxhg8ym0000gn/T/ipykernel_17592/1572979219.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  all_possible = all_possible[additional_filter]


(421, 70, 16.63)

In [32]:
del df_collision["local_authority_ons_district"]

**Location category: LSOA of accident location**

`lsoa_of_accident_location` - lsoa means "lower layer super output area". It is an area defined by the Office of National Statistics (ONS) according to population i.e. each area comprises of between 400 and 1200 households with a usual resident population between 1000 and 3000. These areas fit within local authority boundaries. 

According to the [Road Safety Open Dataset Guide](https://data.dft.gov.uk/road-accidents-safety-data/dft-road-casualty-statistics-road-safety-open-dataset-data-guide-2023.xlsx), this data is for "England and Wales only." so by definition is incomplete, since our data includes Scotland. 

It also contains a lot of missing values.

Since the field is incomplete, we will drop it.

In [33]:
missing = df_collision["lsoa_of_accident_location"] == "-1"
len(df_collision[missing])

4245

In [34]:
del df_collision["lsoa_of_accident_location"]

**Location category: Local authority highway**

Our current dataset has missing `local_authority_highway` categories: 6.85% are missing. 

This is not too high a percentage and perhaps it's possible that more data will fill these gaps. For just now, we drop it.

In [35]:
field_name = "local_authority_highway"
additional_filter = df_collision_key["code/format"] != -1
get_info_on_incomplete_categories(field_name, additional_filter)

/var/folders/bz/vfyjwrvn5lzbwwdbbbxhg8ym0000gn/T/ipykernel_17592/1572979219.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  all_possible = all_possible[additional_filter]


(219, 15, 6.85)

In [36]:
del df_collision["local_authority_highway"]

#### Location category: Police Force

When deleting the coordinates columns, it was mentioned that other location categories existed in the data.

We have just removed some of these possible candidates i.e. `local_authority_ons_district`, `lsoa_of_accident_location` and `local_authority_highway` 

`police_force` is the police force in whose area the accident occured. Police forces handle certain areas of Great Britain and so act as a kind of location category. this category is complete and can be used.

In [37]:
field_name = "police_force"
additional_filter = df_collision_key["note"] != "category discontinued in 2019"
accident_key_field = "label"
get_info_on_incomplete_categories(field_name, additional_filter)

/var/folders/bz/vfyjwrvn5lzbwwdbbbxhg8ym0000gn/T/ipykernel_17592/1572979219.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  all_possible = all_possible[additional_filter]


(44, 0, 0.0)

## Saving reduced dataset

In [38]:
df_collision.head().T

,0,1,2,3,4
police_force,1,1,1,1,1
accident_severity,3,3,3,3,3
number_of_vehicles,1,3,2,2,2
number_of_casualties,1,2,1,1,1
date,2023-01-01 00:00:00,2023-01-01 00:00:00,2023-01-01 00:00:00,2023-01-01 00:00:00,2023-01-01 00:00:00
day_of_week,1,1,1,1,1
time,01:24,02:25,03:50,02:13,01:42
first_road_class,5,6,3,3,3
road_type,2,6,1,6,6
speed_limit,20,30,30,30,30


In [39]:
df_collision.to_csv("./data/collisions1.csv", index=False)